# LLM Prompt Recovery - Final Draftv4

## General idea:


1) If the new and old text are "too similar" then we say it is just "Improve this text" or whatever

2) Use Mistral or something to classify all of the new and old texts

3) Use that information to put it into a cluster

4) From that cluster, generate an average representational prompt

Thanks to Rich Olson for providing some of the formatting code in his Mistral notebook.


In [1]:
# Pip install
!pip install -q -U \
    bitsandbytes peft accelerate \
    transformers unsloth triton xformers torch \
    --no-index \
    --find-links \
    /kaggle/input/prompt-recover-pip-wheels/

In [ ]:
# Imports
import pandas as pd
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
import accelerate
from datasets import load_dataset, Dataset
from huggingface_hub import notebook_login

from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    pipeline,
    DataCollatorWithPadding
)


import numpy as np
#if (not torch.cuda.is_available()): print("Cuda not working/GPU not detected")


2024-04-16 02:58:23.342262: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 02:58:23.342393: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 02:58:23.604485: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Mean prompt setup

In [3]:
response_prefix = "Please improve the following text by reimagining it through the lens of a"
response_suffix = ", retaining the original essence while elevating its clarity, eloquence, and potency by modulating the tone, word choice, and stylistic nuances to harmoniously embody the stylistic features while ensuring the core message remains intact."
base_response="Please improve the following text by reimagining it through the lens of a , retaining the original essence while elevating its clarity, eloquence, and potency by modulating the tone, word choice, and stylistic nuances to harmoniously embody the stylistic features while ensuring the core message remains intact."

# Loading Competition Dataset


In [4]:

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    test_df = pd.read_csv("/kaggle/input/llm-prompt-recovery/test.csv")
else:
    #test_df = pd.read_csv("/kaggle/input/llm-prompt-recovery/test.csv")
    test_df = pd.read_csv("/kaggle/input/gemma-rewrite-nbroad/nbroad-v2.csv")
    #test_df=pd.read_csv("/kaggle/input/combined-results/combined_results_a.csv")
    test_df=test_df.sample(20)
    #test_df['rewritten_text']=test_df['only_rewritten']
    #test_df=test_df.drop(['only_rewritten','has_colon', 'colon_snip_before','colon_snip_after','date','placeholder'],axis=1)





test_df['rewrite_blank']=False
test_df['rewrite_blank']=test_df['rewritten_text'].isna()
test_df['original_text'] = test_df['original_text'].fillna(' ')
test_df['rewritten_text'] = test_df['rewritten_text'].fillna(' ')
testdf_copy=test_df.copy(deep=True)
print(test_df.head())

              id                                      original_text  \
1419  llsUqKWcIj  `` Huh... whats this for?'' I say as I plug th...   
556   XkJfBmJXkI  Death extended her hand to me `` deal?'' she a...   
1879  vWEqkRWQCt  Three men rounded the corner carrying carrying...   
1430  KZkoeukpMZ  I picked up my newspaper. And read the headlin...   
2370  GOlHSsFDJc  “ We are very proud of what you did. Your rese...   

                                         rewrite_prompt  \
1419  Rewrite the story as a modern-day dystopian novel   
556   Rewrite the essay as a poem with strong rhythm...   
1879  Rewrite the essay to make the act deliberate, ...   
1430  Rewrite the story as if they are both doing it...   
2370  Rewrite the story with more humor and visual i...   

                                         rewritten_text  rewrite_blank  
1419  In the dystopian wasteland of the future, huma...          False  
556   Death's hand extended, a deal to make,\nA smil...          False  

In [5]:
original_texts=test_df['original_text'].tolist()
rewritten_texts=test_df['rewritten_text'].tolist()
reslist=[]


## Load Mistral
This is going to use the finetuned unsloth mistral-7b-bnb-4bit

In [6]:
MODEL_NAME = "unsloth/mistral-7b-bnb-4bit"

PEFT_MODEL = "/kaggle/input/mistral-7b-it-bnb-4bit-finetuned-text"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
config = PeftConfig.from_pretrained(PEFT_MODEL)
print(config)
#print(config.base_model_name_or_path)
base_model_path="/kaggle/input/custom-mistral-inference/unsloth"
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(base_model_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)



generation_config = model.generation_config
generation_config.max_new_tokens = 42
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id


generation_config_tt = model.generation_config
generation_config_tt.max_new_tokens = 6
generation_config_tt.temperature = 0.7
generation_config_tt.top_p = 0.7
generation_config_tt.num_return_sequences = 1
generation_config_tt.pad_token_id = tokenizer.eos_token_id
generation_config_tt.eos_token_id = tokenizer.eos_token_id

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='unsloth/mistral-7b-instruct-v0.2-bnb-4bit', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=32, target_modules={'model.layers.31.self_attn.q_proj', 'model.layers.31.self_attn.o_proj', 'model.layers.31.self_attn.k_proj', 'model.layers.31.mlp.gate_proj', 'model.layers.31.mlp.down_proj', 'model.layers.31.mlp.up_proj', 'model.layers.31.self_attn.v_proj'}, lora_alpha=128, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)


/opt/conda/lib/python3.10/site-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


## Chat template Model for Mistral

Here we'll set up the chat templates for text-type inference, colon examples, and prompt rewrite responses.


In [ ]:
chat_template = open('/kaggle/input/mistral-instruct-templ/mistral-instruct.jinja').read()
chat_template = chat_template.replace('    ', '').replace('\n', '')
tokenizer.chat_template=chat_template
print(tokenizer.chat_template)

sys_mes_text_inf="""Context:You are an assistant that is helping me classify types of text. Please give a one or two word response indicating the category type of text. The list of possible words are: ["Article","BlogPost","Diary","Essay","Instructions","Letter","Memos","Newsletters","NewslettersReviews","Poetry","Postcards","Presentations","Profile","Proposal","CinemaScripted","CookingRecipes","Review","Speeches","Storytelling","Tweets"].Use only one category in your reply!"""

sys_mes_prompt_rewrite="""Context:You are an assistant that is helping derive rewrite prompts from the original text and the rewritten text. Please do your best to guess at what the rewrite prompt is from how the original text changes into the rewritten text!"""
device="cuda"



{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'].strip() + '\n\n' %}{% else %}{% set loop_messages = messages %}{% set system_message = '' %}{% endif %}{{ bos_token }}{% for message in loop_messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if loop.index0 == 0 %}{% set content = system_message + message['content'] %}{% else %}{% set content = message['content'] %}{% endif %}{% if message['role'] == 'user' %}{{ '[INST] ' + content.strip() + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ ' '  + content.strip() + ' ' + eos_token }}{% endif %}{% endfor %}


## Prompt for text-type inference
Cluster based approach

In [8]:
cluster_prompts=pd.read_csv("/kaggle/input/clusterprompts/cluster_prompts.csv")
cluster_dict=dict()
for ind,row in cluster_prompts.iterrows():
    cluster_name=row["Cluster"]
    mean_prompt=row["Mean_Prompt"]
    cluster_dict.update({cluster_name:mean_prompt})

## LLM


In [ ]:
base_response="Please improve this text using the writing style with maintaining the original meaning but altering the tone."

In [10]:
def text_inf_mean_prompt(text_type_a,text_type_b):
    try:
        mp=cluster_dict[text_type_b]
    except:
        print(text_type_b)
        print("OH NO!")
        mp=base_response
    return mp

In [11]:
def text_type_m(text):
    text_cat = text.split("[/INST] ")[1]
    text_cat.replace(" ", "")
    best=9999
    best_word="BLAH"
    for word in ["Article","BlogPost","Diary","Essay","Instructions",
                 "Letter","Memos","Newsletters","NewslettersReviews",
                 "Poetry","Postcards","Presentations","Profile","Proposal",
                 "CinemaScripted","CookingRecipes","Review","Speeches",
                 "Storytelling","Tweets"]:
        num = text_cat.find(word)
        if (num < best) and (num>=0):
            best=num
            best_word=word
    return best_word

## Generate approximate prompts

Here we want to efficiently generate approximate prompts for all 3 cases

In [ ]:
messages=[{"role":"system", "content": sys_mes_text_inf}]

for index, row in test_df.iterrows():
    if row['rewrite_blank']:
        result = base_response
        test_df.at[index, 'rewrite_prompt'] = result
        test_df.at[index,'done']=True
        continue
    else:
        original_text=row['original_text']
        rewritten_text=row['rewritten_text']
        
        prompty=f"""Text:{original_text} Category:"""
        new_messages=[{"role":"system", "content": sys_mes_text_inf},{"role":"user","content":prompty}]
        encoding = tokenizer.apply_chat_template(new_messages,tokenize=True, return_tensors="pt").to(device)
        with torch.inference_mode():
          outputs = model.generate(
             encoding,
              generation_config = generation_config_tt
          )
        str_out=tokenizer.decode(outputs[0],skip_special_tokens=True)
        og_m_tt=text_type_m(str_out)
        prompty=f"""Text:{rewritten_text} Category:"""
        new_messages=[{"role":"system", "content": sys_mes_text_inf},{"role":"user","content":prompty}]
        encoding = tokenizer.apply_chat_template(new_messages,tokenize=True, return_tensors="pt").to(device)
        with torch.inference_mode():
          outputs = model.generate(
             encoding,
              generation_config = generation_config_tt
          )
        str_out=tokenizer.decode(outputs[0],skip_special_tokens=True)
        rw_m_tt=text_type_m(str_out)


        if og_m_tt=="Blog post":
            og_m_tt="BlogPost"
        if rw_m_tt=="Blog post":
            rw_m_tt="BlogPost"
        if og_m_tt=="Cinema Scripted":
            og_m_tt="CinemaScripted"
        if rw_m_tt=="Cinema Scripted":
            rw_m_tt="CinemaScripted"
        if og_m_tt=="Recipes":
            og_m_tt="CookingRecipes"
        if rw_m_tt=="Recipes":
            rw_m_tt="CookingRecipes"
        if og_m_tt=="Article":
            og_m_tt="Articles"
        if rw_m_tt=="Article":
            rw_m_tt="Articles"
        if og_m_tt=="Diary":
            og_m_tt="Dairy"
        if rw_m_tt=="Diary":
            rw_m_tt="Dairy"
        if og_m_tt=="Business Proposal":
            og_m_tt="Proposal"
        if rw_m_tt=="Business Proposal":
            rw_m_tt="Proposal"
        if og_m_tt=="Newsletter Reviews":
            og_m_tt="NewslettersReviews"
        if rw_m_tt=="Newsletter Reviews":
            rw_m_tt="NewslettersReviews"
        print(f"{index}")
        print(og_m_tt)
        print(rw_m_tt)
        if rw_m_tt!="BLAH": 
            if og_m_tt!=rw_m_tt:
                result=text_inf_mean_prompt(og_m_tt,rw_m_tt)
                test_df.at[index, 'rewrite_prompt'] = result
                test_df.at[index,'done']=True
                continue
                
        



/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


1419
BLAH
Storytelling


/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


556
Storytelling
Poetry
1879
Storytelling
Storytelling
1430
BLAH
Newsletters
2370
Newsletters
BLAH
2
Letter
Letter
819
Storytelling
Storytelling
2341
Storytelling
Storytelling
1862
Storytelling
BLAH
940
Storytelling
Storytelling
1868
Storytelling
Storytelling
630
Articles
Storytelling
1456
Storytelling
Storytelling
343
Storytelling
Storytelling
194
Newsletters
BLAH
2239
Poetry
BLAH
963
Storytelling
Storytelling
1328
Speeches
Storytelling
2138
Storytelling
Storytelling
483
Articles
Poetry


In [ ]:
#del
del model,tokenizer
torch.cuda.empty_cache()
#
MODEL_NAME = "unsloth/mistral-7b-bnb-4bit"

PEFT_MODEL = "/kaggle/input/mistral-promptguess2/trained-model-it-promptguess-2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
config = PeftConfig.from_pretrained(PEFT_MODEL)
print(config)

base_model_path="/kaggle/input/custom-mistral-inference/unsloth"
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(base_model_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)



generation_config = model.generation_config
generation_config.max_new_tokens = 80
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id


max_new_tokens = 30
max_sentences_in_response = 2

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='unsloth/mistral-7b-instruct-v0.2-bnb-4bit', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=32, target_modules={'model.layers.31.self_attn.q_proj', 'model.layers.31.self_attn.o_proj', 'model.layers.31.self_attn.k_proj', 'model.layers.31.mlp.gate_proj', 'model.layers.31.mlp.down_proj', 'model.layers.31.mlp.up_proj', 'model.layers.31.self_attn.v_proj'}, lora_alpha=128, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)


/opt/conda/lib/python3.10/site-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


In [14]:
chat_template = open('/kaggle/input/mistral-instruct-templ/mistral-instruct.jinja').read()
chat_template = chat_template.replace('    ', '').replace('\n', '')
tokenizer.chat_template=chat_template

In [ ]:
orig_prefix = "Original Text:"


llm_response_for_rewrite = "Provide the new text and I will tell you what new element was added or change in tone was made to improve it - with no references to the original.  I will avoid mentioning names of characters.  It is crucial no person, place or thing from the original text be mentioned.  For example - I will not say things like 'change the puppet show into a book report' - I would just say 'improve this text into a book report'.  If the original text mentions a specific idea, person, place, or thing - I will not mention it in my answer.  For example if there is a 'dog' or 'office' in the original text - the word 'dog' or 'office' must not be in my response."

rewrite_prefix = "Re-written Text:"

response_start = "The request was: "

examples_sequences = [
    (
        "Hey there! Just a heads up: our friendly dog may bark a bit, but don't worry, he's all bark and no bite!",
        "Warning: Protective dog on premises. May exhibit aggressive behavior. Ensure personal safety by maintaining distance and avoiding direct contact.",
        "Please improve the following text by reimagining it through the lens of a warning."
    ),

    (
        "A lunar eclipse happens when Earth casts its shadow on the moon during a full moon. The moon appears reddish because Earth's atmosphere scatters sunlight, some of which refracts onto the moon's surface. Total eclipses see the moon entirely in Earth's shadow; partial ones occur when only part of the moon is shadowed.",
        "Yo check it, when the Earth steps in, takes its place, casting shadows on the moon's face. It's a full moon night, the scene's set right, for a lunar eclipse, a celestial sight. The moon turns red, ain't no dread, it's just Earth's atmosphere playing with sunlight's thread, scattering colors, bending light, onto the moon's surface, making the night bright. Total eclipse, the moon's fully in the dark, covered by Earth's shadow, making its mark. But when it's partial, not all is shadowed, just a piece of the moon, slightly furrowed. So that's the rap, the lunar eclipse track, a dance of shadows, with no slack. Earth, moon, and sun, in a cosmic play, creating the spectacle we see today.",
        "Please improve the following text by reimagining it through the lens of a rap."
    ),
    
    (
        "Drinking enough water each day is crucial for many functions in the body, such as regulating temperature, keeping joints lubricated, preventing infections, delivering nutrients to cells, and keeping organs functioning properly. Being well-hydrated also improves sleep quality, cognition, and mood.",
        "Arrr, crew! Sail the health seas with water, the ultimate treasure! It steadies yer body's ship, fights off plagues, and keeps yer mind sharp. Hydrate or walk the plank into the abyss of ill health. Let's hoist our bottles high and drink to the horizon of well-being!",
        "Please improve the following text by reimagining it through the lens of a pirate would."
    ),
    
    
    (
        "The Johnson family planned a delightful Saturday outing at Green Meadow Park. They packed sandwiches, apples, and iced tea into their picnic basket. The children, Lily and Max, raced each other to the swing set, their laughter echoing across the park. Mr. and Mrs. Johnson followed at a leisurely pace, smiling at the joy of their children. Finding a shaded spot under a grand oak, they spread their blanket and set out the food. After eating, the children kicked a soccer ball around, while their parents lay back, soaking in the serene atmosphere.",
        "Woof! Today's adventure was pawsome! My pack, the Johnsons, led us to the vast expanse of Green Meadow Park, my tail wagging in excitement. They brought the food holder – oh, how I drooled over the scent of sandwiches and apples! My little humans, Lily and Max, dashed towards the swings, and I bounded alongside, barking joyfully. My big humans meandered, allowing me to investigate every intriguing aroma. We claimed our territory under a mighty oak, the perfect spot for naps and guarding. Post-feast, it was playtime! I chased the soccer ball, ensuring my pack's laughter and safety. Life's good when you're the family dog.",
        "Please improve the following text by reimagining it through the lens of a dog."
    ),

            
]


In [ ]:
def remove_numbered_list(text):
    final_text_paragraphs = [] 
    for line in text.split('\n'):
        
        parts = line.split('. ', 1)
        
        if len(parts) > 1 and parts[0].isdigit():
            final_text_paragraphs.append(parts[1].strip())
        else:
            final_text_paragraphs.append(line.strip())

    return ' '.join(final_text_paragraphs)



def trim_to_response(text):
    terminate_string = "[/INST]"
    text = text.replace('</s>', '')
    text = text.replace('"', '')
    text = text.replace("'", '')

    last_pos = text.rfind(terminate_string)
    return text[last_pos + len(terminate_string):] if last_pos != -1 else text


def extract_text_after_response_start(full_text):
    parts = full_text.rsplit(response_start, 1)
    if len(parts) > 1:
        return parts[1].strip()
    else:
        return full_text  
    

def trim_to_first_x_sentences_or_lf(text, x):
    if x <= 0:
        return ""
    text = text.replace("  ", "\n")

    text_chunks = text.split('\n')
    if len(text_chunks)>1:
        return text_chunks[0]+" " + text_chunks[1]
    else:
        return text_chunks[0]


In [17]:
def actually_remove_numbering(text):
    res=text
    for num in ["0","1","2","3","4","5","6","7","8","9"]:
        numbering=num+"."
        res=res.replace(numbering,"")
    for num in ["0","1","2","3","4","5","6","7","8","9"]:
        res=res.replace(num,"")
    return res

In [ ]:
def get_prompt(orig_text, transformed_text):

    messages = []
    messages=[{"role":"system", "content": sys_mes_prompt_rewrite}]
    
    for example_text, example_rewrite, example_prompt in examples_sequences:
        messages.append({"role": "user", "content": f"{orig_prefix} {example_text}"})
        messages.append({"role": "assistant", "content": llm_response_for_rewrite})
        messages.append({"role": "user", "content": f"{rewrite_prefix} {example_rewrite}"})
        messages.append({"role": "assistant", "content": f"{response_start} {example_prompt}"})

    #actual prompt
    messages.append({"role": "user", "content": f"{orig_prefix} {orig_text}"})
    messages.append({"role": "assistant", "content": llm_response_for_rewrite})
    messages.append({"role": "user", "content": f"{rewrite_prefix} {transformed_text}"})
    messages.append({"role": "assistant", "content": f"{response_start} {response_prefix}"})
        
    #give it to Mistral
    model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = model_inputs.to("cuda") 
    generated_ids = model.generate(model_inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)

    
    decoded = tokenizer.batch_decode(generated_ids)
    
    just_response = trim_to_response(decoded[0])        
    
    
    final_text = extract_text_after_response_start(just_response)
    final_text = remove_numbered_list(final_text)
    final_text=actually_remove_numbering(final_text)
    final_text = trim_to_first_x_sentences_or_lf(final_text, max_sentences_in_response)
    
    return final_text

In [ ]:

def combine_prompt(prompt):
    res=prompt.rstrip(".") + response_suffix
    return res

In [20]:
for index, row in test_df.iterrows():
    if row['done']==True:
        continue
    else:
        result = get_prompt(row['original_text'], row['rewritten_text'])
        result = result.rstrip('.') + response_suffix
        test_df.at[index, 'rewrite_prompt'] = result
        
        

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but rig

In [21]:
pd.set_option('display.max_colwidth', None)
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    test_df = test_df[['id', 'rewrite_prompt']]
    test_df.to_csv('submission.csv', index=False)
else:
    test_df.to_csv('submission.csv', index=False)
    testdf_copy.to_csv('copy.csv')
    print(test_df['rewrite_prompt'].head(20))

1419                                                                                                                                                                                                                                                                                                      Please improve this text to be a creative storytelling using the writing style while maintaining the original meaning but altering the tone.
556                                                                                                                                                                                                                                                                                          Please improve this text using the writing style, maintaining the original meaning but altering the tone to expound on its significance in a poetic form.
1879                                                                                             Please improve the follow